# 코루틴(coroutine)
제너레이터를 코루틴으로 활용할 수 있음  
비동기 프로그래밍의 기초를 지원하기 위해 제너레이터가 어떻게 코루틴으로 진화하는 지 살펴보기  

코루틴을 지원하기 위해 추가된 기본 메서드
- .close()
- .throw(ex_type[, ex_value[, ex_traceback]])
- .send(value)

## 제너레이터 인터페이스의 메서드  
위의 코루틴을 위한 메서드가 각각 무엇인지, 어떻게 작동하는지, 어떻게 사용되는 지 확인  
간단한 코루틴 생성  
코루틴의 고급 사용법과 서브 제너레이터(코루틴)에 위임을 통해 리팩토링하는 방법  
여러 코루틴을 조합하는 방법 학습  

### close()
이 메서드를 호출하면 제너레이터에서 GeneratorExit 예외 발생  
GeneratorExit 예외 미처리 시 제너레이터가 더 이상 값을 생성하지 않으며 반복이 중지됨  
- GeneratorExit 예외는 종료 상태를 지정하는 데 사용 가능
- 코루틴이 일종의 자원관리를 하는 경우 이 예외를 통해 코루틴이 보유한 모든 자원 해제 가능 
- 컨텍스트 관리자를 사용하거나 finally 블록에 코드를 배치하는 것과 유사하나 GeneratorExit 예외를 사용하면 보다 명확하게 처리 가능

아래 코드는 코루틴을 사용하여 데이터베이스 연결을 유지한 상태에서 한 번에 모든 레코드를 읽는 대신에 특정 크기의 페이지를 스트리밍함
```python
def stream_db_records(db_handler):
    try:
        while True:
            yield db_handler.read_n_records(10)
            
    except GeneratorExit:
        db_handler.close()
```

```python
>> streamer = stream_db_records(DBHandler("testdb"))
>> next(streamer)
```
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> next(streamer)
```
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> streamer.close()
```
> INFO:...:closing connection to database 'testdb'

이처럼 제너레이터를 호출할  때마다 데이터베이스 핸들러에서 얻은 10개의 레코드를 반환하고, 명시적으로 반복을 끝내고 close()를 호출하면 데이터베이스 연결되 함께 종료됨

### throw(ex_type[, ex_value[, ex_traceback]])
이 메서드는 현재 제너레이터가 중단된 위치에서 예외를 던짐  
제너레이터가 예외를 처리했으면 해당 except 절에 있는 코드가 호출되고, 예외를 처리하지 않았으면 예외가 호출자에게 전파됨  

```python
class CustomerException(Exception):
    pass
    
def stream_data(db_handler):
    while True:
        try:
            yield db_handler.read_n_records(10)
        except CustomException as e:
            logger.warning("처리 가능한 에러 %r, 계속 진행", e)
        except Exception as e:
            logger.error("처리할 수 없는 에러 %r, 중단", e)
            db_handler.close()
            break
```
CustomException을 처리하고 있으며 이 예외가 발생한 경우 제너레이터는 INFO 레벨의 메세지를 기록하고 다음 yield 구문으로 이동하여 데이터베이스에서 다시 데이터를 가져옴  
이 예제에서는 모든 예외를 처리하고 있으나 마지막 블록 (except Exception)이 없으면 제너레이터가 중지된 행에서 예외가 호출자에게 전파되고 제너레이터는 중지됨  


```python
>> streamer = stream_data(DBHandler("testdb"))
>> next(streamer)
```
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> next(streamer)
```
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> streamer.throw(CustomException)
```
> WARNING: 처리 가능한 에러 CustomException(), 계속 진행  
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> streamer.throw(RuntimeError)
```
> ERROR: 처리할 수 없는 에러 RuntimeError(), 중단  
> INFO: 'testdb' 데이터베이스 연결 종료  
> Traceback (most recent call last):  
> ...  
> StopIteration

도메인에서 처리하고 있는 CustomException 예외를 받은 경우 제너레이터는 계속 진행됨  
그러나 그 외 예외는 Exception으로 넘어가서 데이터베이스 연결을 종료하고 반복도 종료하게 됨  
마지막에 StopIteration이 출력된 것에서 알 수 있듯이 이 제너레이터는 이제 더 이상 반복을 할 수 없음

### send(value)
위의 예제에서는 데이터베이스 레코드를 조회하는 제너레이터를 만들고 반복을 끝낼 때 데이터베이스 리소스를 해제함  
-> 제너레이터가 제공하는 close 메서드를 사용하는 좋은 예제이나 코루틴으로 보다 많은 일을 할 수 있음  

예제에서 제너레이트의 주요 기능: 고정된 수의 레코드 읽기  
읽어올 개수를 파라미터로 받아서 호출하도록 수정하려고 해도 next() 함수는 이러한 옵션을 제공하지 않음  
이럴 때 send() 메서드 사용 가능

In [3]:
def stream_db_records(db_handler):
    retrieved_data = None
    previous_page_size = 10
    try:
        while True:
            page_size = yield retrieved_data
            if page_size is None:
                page_size = previous_page_size
            
            previous_page_size = page_size
            
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

이제 send() 메서드를 통해 인자값 전달 가능  
send() 메서드는 **제너레이터와 코루틴을 구분하는 기준이 됨**  
send() 메서드를 사용했다는 것 = yield 키워드가 할당 구문의 오른쪽에 나오게 되고 인자값을 받아서 다른 곳에 할당할 수 있음  

코루틴에서는 일반적으로 다음과 같은 형태로 yield 키워드를 사용함  
```python
receive = yield produced
```

이 경우 yield 키워드는
1. 생성된 값을 호출자에게 보내고 그 곳에 멈추는 것 (호출자는 next() 메서드를 호출해 다음 라운드에서 값을 가져올 수 있음)
2. 거꾸로 호출자로부터 send() 메서드를 통해 전달된 생성된 값을 받는 것 (이렇게 입력된 값은 receive 변수에 할당됨)

코루틴에 값을 전송하는 것은 yield 구문이 멈춘 상태에서만 가능함  
이를 위해서는 코루틴을 해당 상태까지 이동시켜야하는데 해당 상태로 코루틴을 이동시키는 유일한 방법은 next()를 호출하는 것임  
-> **코루틴에게 무엇인가를 보내기 전에 next() 메서드를 적어도 한 번은 호출해야 함을 의미**  

next() 메서드를 호출하지 않으면 아래와 같은 예외 발생 

```python
>>> c = coro()
>>> c.send(1)
```
>Trackback (most recent call last):  
>...  
>TypeError: can't send non-None value to a just-started generator

앞에서 사용한 예제를 데이터베이스에서 읽을 레코드의 길이를 파라미터로 받도록 수정  
1. 제너레이터에서 처음 next()를 호출하면 yield를 포함하는 위치까지 이동하고 현재 상태의 변수 값을 반환하고 거기에 멈춤  
2. 변수의 초기 값이 None이므로 처음 next()를 호출하면 None을 반환  
3-1. 그냥 next()를 호출하면 기본값인 10을 사용하여 평소처럼 이후 작업이 계속됨. next()는 send(None)과 동일  
3-2. send(value)를 통해 명시적인 값을 제공하면 yield 문의 반환값으로 page_size 변수에 설정됨. 사용자가 지정한 값이 page_size로 설정됨

이후 호출에서도 동일한 로직이 적용되므로 아무 떄나 페이지 크기를 지정할 수 있게 됨  

위와 같은 사항을 반영하여 깔끔하게 코드를 고치면 아래와 같음

In [5]:
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

이 코드도 기대한 대로 잘 동작하나 역시 send() 전에 next()를 먼저 호출해야 함 -> 호출하지 않으면 TypeError 발생  

next()를 반드시 호출해야 한다는 것을 기억할 필요 없이 코루틴을 생성하자마자 바로 사용할 수 있다면 훨씬 편할 것임  
아래 @prepare_coroutine 데코레이터는 코루틴을 좀 더 편리하게 사용하도록 자동으로 초기화를 해줌

In [7]:
def prepare_coroutine(coroutine):
    def wrapped(*args, **kwargs):
        advanced_coroutine = coroutine(*args, **kwargs)
        next(advanced_coroutine)
        return advanced_coroutine

    return wrapped

@prepare_coroutine
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

```python
>> streamer = stream_db_records(DBHandler("testdb"))
>> len(streamer.send(5))
```
>5

## 코루틴 고급 주제  
많은 코루틴을 처리하다 보면 애플리케이션의 로직이 복잡해짐에 따라 예외처리, 다양한 곳에서의 서브코루틴 값 사용, 코루틴 스케쥴링 등을 해야한다는 필요가 생김  
이러한 일을 더 간편하게 하기 위해서는 제너레이터를 더 확장해야하는데, 이는 yield from 이라는 생성자 구문을 도입하여 해결할 수 있음

### 코루틴에서 값 반환하기  
반복이란 StopIteration 예외가 발생할 때까지 next() 메서드를 계속해서 호출하는 메커니즘  
제너레이터에서는 일반적으로 한 번에 하나씩 값을 생성하며 for 루프의 모든 단계에서 생성되는 각 값에만 시경을 씀  
그러나 코루틴은 기술적으로는 제너레이터이나 반복을 염두에 두고 만든 것이 아니라 나중에 코드가 실행될 때까지 코드의 실행을 멈추는 것을 목표로 함  

**코루틴은 일반적으로 반복보다는 상태를 중단하는 데 초점을 맞춤**  
코루틴을 사용해 정보를 처리하고 실행을 일시 중단하는 경우 경량 스레드라고 생각하는 것이 좋음  
제너레이터에서 값을 반환(return)하면 반복이 즉시 중지되어 더 이상 반복을 할 수 없음  
StopIteration 예외가 발생해도 예외 객체 내에 반환 값이 저장되어 있음  

아래 예제는 제너레이터를 사용해 두 개의 값을 생성하고 세 번째 값을 반환함  
마지막 return 되는 값을 구하기 위해 예외를 처리하는 방법과 예외에서 어떻게 값을 구하는 지 유의

In [9]:
def generator():
    yield 1
    yield 2
    return 3

value = generator()
print(next(value))
print(next(value))
try:
    print(next(value))
except StopIteration as e:
    print(">>>>> return value ", e.value)
    

1
2
>>>>> return value  3


## 작은 코루틴에 위임하기 - yield from 구문
제너레이터가 값을 반환할 수는 있었지만 value = generator() 와 같은 문장은 동작하지 않았음  
그러나 value = yield from generator() 와 같이 작성하면 가능함  

### 가장 간단한 yield from 사용 예
가장 간단한 형태의 yield from 구문은 제너레이터 체인에서 살펴볼 수 있음  
제너레이터 체인은 여러 제너레이터를 하나의 제너레이터로 합치는 기능을 하는데 중첩된 for 루프를 사용해 하나씩 모으는 대신에 서브 제너레이터의 값을 한 번에 수집할 수 있게 해줌  

아래 코드는 표준 라이브러리인 itertools.chain()과 비슷한 함수로 여러 개의 이터러블을 받아서 하나의 스트림으로 반환함

In [10]:
def chain(*iterables):
    for it in iterables:
        for value in it:
            yield value

여러 이터러블을 받아서 모두 이용함  
모두 이터러블이므로 for 구문을 지원하여 개별 값(value)을 구하고 싶다면 중첩 루프를 사용하면 됨  
이렇게 되면 리스트를 튜플과 비교하는 것처럼 직접 비교가 어려운 자료형에 대해서도 한 번에 처리할 수 있음

여기서 yield from 구문을 사용하면 서브 제너레이터에서 직접 값을 생산할 수 있으므로 중첩 루프를 피할 수 있음  
yield from 구문을 사용해 다음과 같이 코드를 단순화 할 수 있음

In [11]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [12]:
list(chain("hello", ["world"], ("tuple", "of", "values")))

['h', 'e', 'l', 'l', 'o', 'world', 'tuple', 'of', 'values']

yield from 구문은 어떤 이터러블에 대해서도 동작하며 이것을 사용하면 마치 최상위 제너레이터가 직접 값을 yield한 것과 같은 효과를 나타냄  

yield from 은 어떤 형태의 이터러블에서도 동작하므로 제너레이터 표현식도 마찬가지임  
아래는 yield from 구문을 활용해 입력된 파라미터의 모든 제곱 지수를 만드는 제너레이터임

In [13]:
def all_powers(n, pow):
    yield from (n ** i for i in range(pow+1))

# all_powers(2,3) 은 2^0, 2^1, 2^2, 2^3 생산

In [15]:
list(all_powers(2, 3))

[1, 2, 4, 8]

이렇게 하면 기존의 서브 제너레이터에서 for문을 사용해 값을 생산하는 대신 한 줄로 직접 값을 생산할 수 있으므로 편리함  
그러나 이것만 가지고 yield from을 언어에서 지원해야 하는 이유로 꼽기에는 어려움  
사실 위와 같은 동작은 의도하지 않은 부가적인 효과이며 yield from의 진짜 존재 이유는 아래의 두가지 이유가 가장 큼  
1. 서브 제너레이터에서 반환된 값 구하기
1. 서브 제너레이터와 데이터 송수진하기

### 서브 제너레이터에서 반환된 값 구하기  
아래 예제는 수열을 생산하는 두 개의 중첩된 제너레이터를 호출함  
각각의 제너레이터는 값을 반환하는데 최상위 제너레이터는 쉽게 반환값을 확인할 수 있음 -> yield from 구문을 사용했기 때문

In [21]:
def sequence(name, start ,end):
    print("%s 제너레이터 %i에서 시작" %(name, start))
    yield from range(start, end)
    print("%s 제너레이터 %i에서 종료" %(name, end))
    return end

def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)
    return step1 + step2

In [28]:
g = main()
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

first 제너레이터 0에서 시작
0
1
2
3
4
first 제너레이터 5에서 종료
second 제너레이터 5에서 시작
5
6
7
8
9
second 제너레이터 10에서 종료


StopIteration: 15

main 제너레이터의 첫 번째 행은 내부 제너레이터로 위임하여 생산된 값을 가져옴  
sequence 제너레이터 종료시의 반환값을 step1로 받아와서 다음 sequence 제너레이터에 전달함  
두번째 제너레이터 역시 종료시 값(10)을 반환하고 그러면 main 제너레이터는 이 두 결과의 합(5+10=15)을 반환함  
이 값은 StopIteration에 포함된 값임  
**yield from을 사용하면 코루틴 종료 시 최종 반환 값을 구할 수 있음**

### 서브 제너레이터와 데이터 송수신하기
제너레이터는 코루틴처럼 동작할 수 있음  
값을 전송하고 예외를 던제면 코루틴 역할을 하는 해당 제너레이터는 값을 받아서 처리하거나 반드시 예외를 처리해야 함  
앞의 예제처럼 서브 제너레이터에 위임한 코리틴에 대해서도 마찬가지임  
그런데 수동으로 이런 것들을 처리하면 복잡할 것이므로 yield from 에서 자동으로 처리하지 않을 경우의 직접 처리하는 코드는 PEP-380에서 확인 가능  

앞의 예제에서 최상의 main 제너레이터는 그대로 유지하고, 값을 수신하고 예외를 처리할 내부 제너레이터인 sequence 함수를 수정  

In [35]:
class CustomException(Exception):
    pass

def sequence(name, start, end):
    value = start
    print("%s 제너레이터 %i에서 시작" %(name, start))
    while value < end:
        try:
            received = yield value
            print("%s 제너레이터 %r 값 수신" %(name, received))
            value += 1
        except CustomException as e:
            print("%s 제너레이터 %s 에러 처리" %(name, e))
            received = yield "OK"
    return end

In [40]:
g = main()
print(next(g))
print("------------------------")
print(next(g))
print("------------------------")
print(g.send("첫 번째 제너레이터를 위한 인자 값"))
print("------------------------")
print(g.throw(CustomException("처리 가능한 예외 던지기")))
print("------------------------")
print(next(g))
print("------------------------")
print(next(g))
print("------------------------")
print(next(g))
print("------------------------")
print(next(g))
print("------------------------")
print(g.throw(CustomException("두 번째 제너레이터를 향한 예외 던지기")))

first 제너레이터 0에서 시작
0
------------------------
first 제너레이터 None 값 수신
1
------------------------
first 제너레이터 '첫 번째 제너레이터를 위한 인자 값' 값 수신
2
------------------------
first 제너레이터 처리 가능한 예외 던지기 에러 처리
OK
------------------------
2
------------------------
first 제너레이터 None 값 수신
3
------------------------
first 제너레이터 None 값 수신
4
------------------------
first 제너레이터 None 값 수신
second 제너레이터 5에서 시작
5
------------------------
second 제너레이터 두 번째 제너레이터를 향한 예외 던지기 에러 처리
OK


이 예제에서는 sequence 서브 제너레이터에 값을 보내지 않고 오직 main 제너레이터에 값을 보냈는데, 실제 값을 받는 것은 내부 제너레이터임  
명시적으로 sequence에 값을 보낸 적은 없지만 실질적으로는 yield from을 통해 sequence에 데이터를 전달한 것  

main 코루틴은 내부적으로 두 개의 다른 코루틴을 호출하여 값을 생산하며 특정 시점에서 보면 둘 중 하나는 멈춰져 있는 상태임  
로그를 통해 첫 번째 코루틴이 멈춰진 상태에서 데이터를 전송해도 첫 번째 코루틴 인스턴스가 값을 받음을 알 수 있음  
예외를 던질 때도 첫번째 코루틴이 예외를 받음
첫 번째 코루틴이 끝나면 step1 변수에 값을 반환하고 그 값을 두 번째 코루틴에 입력으로 전달함

각 코루틴이 생성하는 값 역시 특정 단계에서 send()를 호출했을 때 생성하는 값은 현재 main 제너레이터가 멈춰있던 서브 코루틴에서 생성한 값임  
처리 가능한 예외를 던지면 sequence 코루틴에서 OK를 생성하며 호출자 코루틴인 main에 전파함